### Problem Set #7
#### MACS 30100, Dr. Evans
#### Name: Xiang Zhang
#### snzhang@uchicago.edu

In [111]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from pylab import rcParams
import warnings
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.cross_validation import train_test_split
from sklearn import metrics 
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

In [112]:
auto = pd.read_csv("https://raw.githubusercontent.com/UC-MACSS/persp-model_W18/master/ProblemSets/PS4/Auto.csv", na_values = "?")
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [113]:
auto['const'] = 1
auto = auto.dropna()
auto = auto.drop(['name'], 1)
auto['horsepower'] = auto['horsepower'].convert_objects(convert_numeric = True)
auto['mpg_high'] = auto['mpg'].apply(lambda x: 1 if x >= auto['mpg'].median() else 0).astype('category')

xvars = ['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin', 'const']
yvar = 'mpg_high'

Xvars = auto[xvars].values
yvals = auto[yvar].values

C:\Users\Xiang Zhang\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: convert_objects is deprecated.  Use the data-type specific converters pd.to_datetime, pd.to_timedelta and pd.to_numeric.
  after removing the cwd from sys.path.


In [114]:
k = 4
kf = KFold(n_splits=k, random_state = 15, shuffle=True)
kf.get_n_splits(Xvars)
Log_error_0 = np.zeros(k)
Log_error_1 = np.zeros(k)
Log_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    # Get training and test set
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    
    LogReg = LogisticRegression()
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    error = y_test != y_pred
    error_all_class = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum() 
    Log_error_0[k_ind] = error_0
    Log_error_1[k_ind] = error_1
    Log_MSE[k_ind] = error_all_class
    
    print('k =', k_ind, '\n',classification_report(y_test, y_pred))
    print('Error rate for category 0 is', error_0, ', Error rate for category 1 is', error_1)
    print('The MSE of the model is', error_all_class)
    
    k_ind = k_ind + 1

MSE_kf = Log_MSE.mean()
MSE_kf_std = Log_MSE.std()
print('\n')
print('test estimate MSE k-fold=', MSE_kf,
      'test estimate MSE standard err=', MSE_kf_std)

print('The average error rate category 0 and std are', Log_error_0.mean(), Log_error_0.std())
print('The average error rate category 1 and std are', Log_error_1.mean(), Log_error_1.std())

k = 0 
              precision    recall  f1-score   support

          0       0.94      0.89      0.92        55
          1       0.87      0.93      0.90        43

avg / total       0.91      0.91      0.91        98

Error rate for category 0 is 0.0576923076923 , Error rate for category 1 is 0.130434782609
The MSE of the model is 0.0918367346939
k = 1 
              precision    recall  f1-score   support

          0       0.88      0.91      0.90        47
          1       0.92      0.88      0.90        51

avg / total       0.90      0.90      0.90        98

Error rate for category 0 is 0.122448979592 , Error rate for category 1 is 0.0816326530612
The MSE of the model is 0.102040816327
k = 2 
              precision    recall  f1-score   support

          0       0.85      0.87      0.86        45
          1       0.88      0.87      0.88        53

avg / total       0.87      0.87      0.87        98

Error rate for category 0 is 0.152173913043 , Error rate for category 

#### (b) Random Forest

In [115]:
Xvars = auto[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values

Random_forest = RandomForestClassifier(n_estimators = 20, max_features=2, bootstrap=True,
                                       oob_score=True, random_state=25)
Random_forest.fit(Xvars, yvals)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=True, random_state=25, verbose=0, warm_start=False)

In [116]:
ypred = Random_forest.oob_decision_function_.T[1]

pred = []
for item in ypred:
    if item >= 0.5:
        pred.append(1)
    else:
        pred.append(0)

pred = np.array(pred)

MSE_fullsample = mean_squared_error(pred, yvals)
        
print("The MSE for the full sample is", MSE_fullsample)

pred = pred.tolist()
pred_0 = []
yval_0 = []
pred_1 = []
yval_1 = []
for i, item in enumerate(pred):
    if item < 0.5:
        pred_0.append(0)
        yval_0.append(yvals[i])
    else:
        pred_1.append(1)
        yval_1.append(yvals[i])

pred_0 = np.array(pred_0)
pred_1 = np.array(pred_1)
yval_0 = np.array(yval_0)
yval_1 = np.array(yval_1)

MSE_0 = mean_squared_error(pred_0, yval_0)
print("The MSE for the 0 category is", MSE_0)

MSE_1 = mean_squared_error(pred_1, yval_1)
print("The MSE for the 1 category is", MSE_1)

The MSE for the full sample is 0.0714285714286
The MSE for the 0 category is 0.0578947368421
The MSE for the 1 category is 0.0841584158416


#### (3) SVM

In [117]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvars)
SVM_error_0 = np.zeros(k)
SVM_error_1 = np.zeros(k)
SVM_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    
    # Train model
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    error = y_test != y_pred
    error_all_class = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum()
    
    SVM_error_0[k_ind] = error_0
    SVM_error_1[k_ind] = error_1
    SVM_MSE[k_ind] = error_all_class
    
    print('k =', k_ind, '\n',classification_report(y_test, y_pred))
    if (y_pred == 0).sum() != 0:
        print('error rate for category 0 is: ', error_0)
    else:
        print('error rate for category 0 is: NA')
    
    if (y_pred == 1).sum() != 0:
        print('error rate for category 1 is: ', error_1)
    else:
        print('error rate category 1 is: NA')
    
    print('The MSE of the model is', error_all_class)
    k_ind = k_ind + 1

MSE_SVM = SVM_MSE.mean()
MSE_SVM_std = SVM_MSE.std()
print('\n')
print('test estimate MSE k-fold=', MSE_SVM,
      'test estimate MSE standard err=', MSE_SVM_std)

print('The average error rate category 0 and std are', SVM_error_0.mean(), SVM_error_0.std())
SVM_error_1 = SVM_error_1[[0, 2, 3]]
print('The average error rate category 1 and std are', SVM_error_1.mean(), SVM_error_1.std())

k = 0 
              precision    recall  f1-score   support

          0       1.00      0.04      0.07        55
          1       0.45      1.00      0.62        43

avg / total       0.76      0.46      0.31        98

error rate for category 0 is:  0.0
error rate for category 1 is:  0.552083333333
The MSE of the model is 0.540816326531
k = 1 
              precision    recall  f1-score   support

          0       0.48      1.00      0.65        47
          1       0.00      0.00      0.00        51

avg / total       0.23      0.48      0.31        98

error rate for category 0 is:  0.520408163265
error rate category 1 is: NA
The MSE of the model is 0.520408163265
k = 2 
              precision    recall  f1-score   support

          0       0.47      1.00      0.64        45
          1       1.00      0.04      0.07        53

avg / total       0.76      0.48      0.33        98

error rate for category 0 is:  0.53125
error rate for category 1 is:  0.0
The MSE of the model is

C:\Users\Xiang Zhang\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\Xiang Zhang\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


#### (d)
Based on the results of the three models:
- The random forest model has the lowest error rates for average MSE, and for category 0, category 1. 
- SVM model has the highest error rates for average MSE, and for category 0, category 1.
- The logit model is in between

So random forest model is the best predictor